<a href="https://colab.research.google.com/github/toomahawkk/Felix-Code/blob/master/FelixMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import torch.nn.functional as F
import math
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
from sklearn import metrics
import torch
import itertools
from torchvision import models
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import nn
from torch.utils.data import Dataset, DataLoader
import os
from torch.nn import MaxPool2d
import chainer.links as L
from PIL import Image
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
plt.ion()

#Transforming the input data (Tensor, Normalize)
train_set = torchvision.datasets.MNIST(root='./data/MNIST', train=True, download=True, transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=(0.5,), std=(0.5,))]))

#Setting DataLoader batch size
train_loader = torch.utils.data.DataLoader(train_set, batch_size = 10)
torch.set_printoptions(linewidth=120)

Checking how many files & overall label distribution:

In [0]:
print(len(train_set))
train_set.train_labels.bincount()

Show single images (from train_set):

In [0]:
sample = next(iter(train_set))
print(type(sample))

image, label = sample

plt.imshow(image.squeeze(),cmap = 'gray')
print("label",label)

Show batch images (from train_loader):

In [0]:
batch = next(iter(train_loader))

image, label = batch

grid = torchvision.utils.make_grid(image, nrow=5)

plt.imshow(np.transpose(grid,(1,2,0)))
print(label)


Now we create the CNN:

In [0]:
class Net(nn.Module):    
    def __init__(self):
        super(Net, self).__init__()
          
        self.features = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5, stride=1),
            nn.BatchNorm2d(10),
            nn.ReLU(inplace=True),
            nn.Conv2d(10, 20, kernel_size=5, stride=1),
            nn.BatchNorm2d(20),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
          
        self.classifier = nn.Sequential(
            nn.Dropout(p = 0.5),
            nn.Linear(10 * 10 * 20, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(512, 10),
        )
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = x.view(-1, x.size(1))
        x = self.classifier(x)
        
        return x     

Step 1

Clearing the Gradients => optimizer.zero_grad()
Clear out the gradients accumulated for the parameters of the network before calling loss.backward() and optimizer.step()

Step 2

Compute the loss => criterion( predicted_target, target)
Compute the loss between the predicted value and the target value within the loss function previously defined

Step 3

Backpropogation => loss.backward()
Back-prop all the layers in all the layers of the network

Step 4

Taking an optimization step => optimizer.step()
Update the parameters of the network



In [0]:
model = Net()

optimizer = optim.Adam(model.parameters(), lr=0.0001)

criterion = nn.CrossEntropyLoss()

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    

In [0]:
def train(epoch):
    model.train()
    exp_lr_scheduler.step()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        #print(data.size())
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        #print(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        
        loss.backward()
        optimizer.step()
        
        if (batch_idx + 1)% 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
                100. * (batch_idx + 1) / len(train_loader), loss.item()))

In [0]:
def evaluate(data_loader):
    model.eval()
    loss = 0
    correct = 0
    
    for data, target in data_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        output = model(data)
        loss += F.cross_entropy(output, target, size_average=False).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    loss /= len(data_loader.dataset)
        
    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))

**RUN** **THE** **CNN**!!!!:

In [95]:
n_epochs = 5

for epoch in range(n_epochs):
    train(epoch)
    evaluate(train_loader)

Train Epoch: 0 [1000/60000 (2%)]	Loss: 1.545456
Train Epoch: 0 [2000/60000 (3%)]	Loss: 1.516000
Train Epoch: 0 [3000/60000 (5%)]	Loss: 0.993577
Train Epoch: 0 [4000/60000 (7%)]	Loss: 0.728333
Train Epoch: 0 [5000/60000 (8%)]	Loss: 0.557907
Train Epoch: 0 [6000/60000 (10%)]	Loss: 0.750769
Train Epoch: 0 [7000/60000 (12%)]	Loss: 0.712131
Train Epoch: 0 [8000/60000 (13%)]	Loss: 0.650096
Train Epoch: 0 [9000/60000 (15%)]	Loss: 0.553767
Train Epoch: 0 [10000/60000 (17%)]	Loss: 0.278427
Train Epoch: 0 [11000/60000 (18%)]	Loss: 0.585858
Train Epoch: 0 [12000/60000 (20%)]	Loss: 0.422710
Train Epoch: 0 [13000/60000 (22%)]	Loss: 0.477526
Train Epoch: 0 [14000/60000 (23%)]	Loss: 0.456798
Train Epoch: 0 [15000/60000 (25%)]	Loss: 0.347398
Train Epoch: 0 [16000/60000 (27%)]	Loss: 0.258189
Train Epoch: 0 [17000/60000 (28%)]	Loss: 0.216747
Train Epoch: 0 [18000/60000 (30%)]	Loss: 0.251738
Train Epoch: 0 [19000/60000 (32%)]	Loss: 0.259760
Train Epoch: 0 [20000/60000 (33%)]	Loss: 0.173400
Train Epoch: 0